In [1]:
import numpy as np  
from layers import *
import torch
from torch import nn
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets
from sklearn.metrics import accuracy_score

Here some some useful links that help me understand how to build a convolutional layer:  
[Standford Intro](https://cs231n.github.io/convolutional-networks/#conv)  
[conv1d,2d,3d difference](https://datascience.stackexchange.com/questions/51470/what-are-the-differences-between-convolutional1d-convolutional2d-and-convoluti)

Fully Connected Layer Test

In [2]:

# iris = datasets.load_iris()
# X = iris.data# we only take the first two features.
# y = iris.target
# Y = np.zeros((y.size, 3))
# Y[np.arange(y.size),y] = 1

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



# layers = [Dense(X.shape[1], 10), Relu(), Dense(10,3),SoftMax()]


# max_ite = 100
# alpha = 0.0001

# for i in range(1000):
#     train_X = X_train
#     for layer in layers:
#         train_X = layer.forward(train_X)
#         #print(train_X)
#     E = (train_X - y_train) * alpha
#     for k in range(len(layers)-1,-1,-1):
#         E = layers[k].backward(E)
#     #print(E)
# for layer in layers:
#     X_test = layer.forward(X_test)
# y_pred = toPredict(X_test)
# accuracy_score(y_test, y_pred)

Test if feedforward portion of conv2d is correct.  
I use torch.conv2d as a reference implementation

In [3]:
np.random.seed(0)
initial_weights = np.random.standard_normal((5,3,2,2))
initial_biases = np.random.standard_normal((5,))
img_h = 10
img_w = 10
sample_input = np.random.random_sample((2,3,img_h,img_w))
conv_ref = nn.Conv2d(3,5,2,stride=1)
conv_ref.weight = nn.Parameter(torch.tensor(initial_weights))
conv_ref.bias   = nn.Parameter(torch.tensor(initial_biases))
conv_imp = Conv2D(3,5,2,2,1)
conv_imp.filters = initial_weights
conv_imp.biases  = initial_biases
ref_result = conv_ref(torch.tensor(sample_input)).cpu().detach().numpy()
imp_result = conv_imp.forward(sample_input)
diff_count = np.sum(~np.isclose(ref_result, imp_result,rtol=0.001))
if diff_count == 0:
    print("conv feedforward looks good!")

conv feedforward looks good!


In [4]:
np.random.seed(0)
initial_weights = np.random.standard_normal((5,3,2,2))
initial_biases = np.random.standard_normal((5,))
img_h = 10
img_w = 10
sample_input = np.random.random_sample((2,3,img_h,img_w))
# print(sample_input)
conv_ref = nn.Conv2d(3,5,2,stride=1)
conv_ref.weight = nn.Parameter(torch.tensor(initial_weights))
conv_ref.bias   = nn.Parameter(torch.tensor(initial_biases))
conv_imp = Conv2D(3,5,2,2,1)
conv_imp.filters = initial_weights
conv_imp.biases  = initial_biases
ref_result = conv_ref(torch.tensor(sample_input))
imp_result = conv_imp.forward(sample_input)

y_true = np.random.random_sample((2,5,9,9))
y_true_t = torch.tensor(y_true)
mse = nn.MSELoss()
loss = mse(ref_result, y_true_t)/2
loss.backward()
optim = torch.optim.SGD(conv_ref.parameters(), lr=1)
optim.step()
loss = (imp_result - y_true)/ np.prod(imp_result.shape)
x = conv_imp.backward(loss)
ref_weights = conv_ref.weight.cpu().detach().numpy()
diff_count = np.sum(~np.isclose(ref_weights,conv_imp.filters,rtol=0.01))
# print(ref_weights)
# print(conv_imp.filters)
if diff_count ==0 :
    print("backpropagation of conv2d looks good")
else:
    print(diff_count)

backpropagation of conv2d looks good


Test if feedforward of max pooling is correct

In [5]:
sample_input = np.random.random_sample((2,3,10,10))
maxpool_ref = nn.MaxPool2d(2,1)
result_ref = maxpool_ref(torch.tensor(sample_input))
maxpool_imp = MaxPooling(2,1)
result_imp = maxpool_imp.forward(sample_input)
diff_count = np.sum(~np.isclose(result_ref, result_imp))
if diff_count == 0:
    print("maxpool feedforward looks good")


maxpool feedforward looks good


In [6]:
p = MaxPooling(2,2)
forward_out = p.forward(x[0:1000])
back_out = p.backward(forward_out)
print(back_out.shape)
print("done")

In [7]:
data = np.load("mnist.npy")
y = data[:, 784:785]
y = OneHotEncoder().fit_transform(y).toarray()

x = data[:,0:784]/256
x = x.reshape((70000,1,28,28))
X_train = x[0:50]
y_train = y[0:50]
X_test = x[1000:1100]
y_test = y[1000:1100]
n = len(X_train)
layers = [Conv2D(1,5,2,2,1),Relu(),MaxPooling(2,2),Flatten(),Dense(845,10),SoftMax()]
max_ite = 100
alpha = 0.0001

In [8]:
for i in range(10):
    
    train_X = X_train
    for layer in layers:
#         print(layer)
#         print(train_X.shape)
        train_X = layer.forward(train_X)
        #print(train_X)
    E = (train_X - y_train)/n * alpha
    for k in range(len(layers)-1,-1,-1):
        E = layers[k].backward(E)
    print(E)
for layer in layers:
    X_test = layer.forward(X_test)
y_pred = toPredict(X_test)
accuracy_score(y_test, y_pred) # xie


[[[[ 6.36226045e-07  1.13178534e-06  5.51671353e-06 ... -4.24377287e-06
     0.00000000e+00  0.00000000e+00]
   [ 7.76083255e-07  2.61213733e-06 -1.51872678e-06 ... -1.04535667e-05
     0.00000000e+00  0.00000000e+00]
   [-2.37984986e-06  1.10070945e-06 -1.62864012e-06 ... -5.60456147e-06
     0.00000000e+00  0.00000000e+00]
   ...
   [ 1.95529097e-06  8.00264792e-06  6.96748077e-06 ... -4.15151767e-06
     0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00 -2.97437032e-07 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]]]


 [[[-1.81123594e-06 -2.13292807e-06  5.78291230e-06 ...  2.38934200e-06
     0.00000000e+00  0.00000000e+00]
   [-2.72025992e-06 -4.25664986e-07 -4.62870176e-06 ...  5.93952487e-06
     0.00000000e+00  0.00000000e+00]
   [ 1.43472856e-06  1.60629426e-06 -3.79303783e-06 ...  4.78392000e-06
     0.00000000e+00  0.00000000e+00]
   ...
 

KeyboardInterrupt: 